In [1]:
import whatimage
import pyheif
from PIL import Image
import exifread
import json
from PIL.ExifTags import TAGS
from PIL.ExifTags import GPSTAGS
import io
import datetime
from tqdm import tqdm

import os
import folium
import geopandas as gpd
import earthpy as et


def get_geotagging(exif):
    if not exif:
        raise ValueError("No EXIF metadata found")

    geotagging = {}
    for (idx, tag) in TAGS.items():
        if tag == 'GPSInfo':
            if idx not in exif:
                raise ValueError("No EXIF geotagging found")

            for (key, val) in GPSTAGS.items():
                if key in exif[idx]:
                    geotagging[val] = exif[idx][key]

    return geotagging

def get_decimal_from_dms(dms, ref):

    degrees = dms[0][0] / dms[0][1]
    minutes = dms[1][0] / dms[1][1] / 60.0
    seconds = dms[2][0] / dms[2][1] / 3600.0

    if ref in ['S', 'W']:
        degrees = -degrees
        minutes = -minutes
        seconds = -seconds
        

    return round(degrees + minutes + seconds, 5)

def get_coordinates(geotags):
    #print (geotags['GPSLatitude'], geotags['GPSLatitudeRef'])
    #print (geotags['GPSLongitude'], geotags['GPSLongitudeRef'])
    lat = get_decimal_from_dms(geotags['GPSLatitude'], geotags['GPSLatitudeRef'])

    lon = get_decimal_from_dms(geotags['GPSLongitude'], geotags['GPSLongitudeRef'])
    
    return (lat, lon)


def get_labeled_exif(exif):
    labeled = {}
    for (key, val) in exif.items():
        labeled[TAGS.get(key)] = val

    return labeled

def read_heic(path):
    with open(path, 'rb') as file:
        image = pyheif.read_heif(file)
        for metadata in image.metadata or []:
            if metadata['type'] == 'Exif':
                fstream = io.BytesIO(metadata['data'][6:])

    # now just convert to jpeg
    #pi = Image.open(fstream)
    #pi.save(f"{path}.jpg".replace("heic", "jpeg"), "JPEG")

    # or do EXIF processing with exifread
    tags = exifread.process_file(fstream)
    
    return tags


def getTimeLocation(path):
    
    with open(path, 'rb') as f:
        data = f.read()

        fmt = whatimage.identify_image(data)
    
        #print (fmt)
    
        if fmt in ['heic', 'avif']:
            exif_data = read_heic(path)
            
            geotags = get_geotagging(exif_data)
            print(get_coordinates(geotags))
        else:
            img = Image.open(path)
            exif_data = img._getexif()
            
            #print (get_labeled_exif(exif_data))
            
            geotags = get_geotagging(exif_data)
            
            time_location = {"location": get_coordinates(geotags), "time": datetime.datetime.strptime(get_labeled_exif(exif_data)["DateTimeOriginal"], '%Y:%m:%d %H:%M:%S')}
            
            return time_location


#decodeImage(photo_2)

def plot_on_map (pics):
    map_osm = folium.Map(location=[33.73692, 132.48821])
    
    for pic in pics:

        folium.Marker(pics[pic]["metadata"]["location"], popup=pic).add_to(map_osm)

    return map_osm


In [2]:
photo_2 = "./IMG_0004.jpeg"
print (getTimeLocation(photo_2))

{'location': (34.9764, 138.46699), 'time': datetime.datetime(2019, 12, 30, 16, 12, 52)}


In [3]:
# You may need to restart your runtime prior to this, to let your installation take effect
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import cv2
import random


# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog

from detectron2.data import MetadataCatalog
category_name = MetadataCatalog.get("coco_2017_val").thing_classes

category_name[:10]

['person',
 'bicycle',
 'car',
 'motorcycle',
 'airplane',
 'bus',
 'train',
 'truck',
 'boat',
 'traffic light']

In [4]:


im = cv2.imread(photo_2)
cv2.imshow("test", im)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [5]:
def process_foto (filename):
    im = cv2.imread(filename)
        
    cfg = get_cfg()
    # add project-specific config (e.g., TensorMask) here if you're not running a model in detectron2's core library
    cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # set threshold for this model
    # Find a model from detectron2's model zoo. You can use the https://dl.fbaipublicfiles... url as well
    cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
    predictor = DefaultPredictor(cfg)
    outputs = predictor(im)

    labels = [category_name[x] for x in outputs["instances"].pred_classes.tolist()]

    metadata = getTimeLocation(filename)
    

    v = Visualizer(im[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1)
    v = v.draw_instance_predictions(outputs["instances"].to("cpu"))


    cv2.imwrite(filename + "_detectroned.jpeg", v.get_image()[:, :, ::-1])

    return {"labels": labels, "metadata": metadata}
        
        

In [6]:
process_foto("IMG_0924.jpeg")

{'labels': ['car', 'person'],
 'metadata': {'location': (35.08654, 138.85692),
  'time': datetime.datetime(2019, 12, 29, 13, 50, 50)}}

In [9]:
import os


photo_folders = "./"

database = {}

for filename in os.listdir(photo_folders):
    if filename.endswith(".jpg") or filename.endswith(".jpeg"):
        print ("processing:", filename)
        database[filename] = process_foto(filename)

processing: IMG_4420.jpeg
processing: IMG_1439.jpeg
processing: IMG_1903.jpeg
processing: IMG_0924.jpeg
processing: IMG_0004.jpeg
processing: IMG_1743.jpeg


In [10]:
def search_object(query, odatabase):
    result = {}
    for file in odatabase:
        if query.lower() in odatabase[file]["labels"]:
            result[file] =  odatabase[file]
    return result

In [16]:
cats =  (search_object("cat", database))

In [17]:
plot_on_map(cats)